# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import * 

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=guest password=guest123")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = 'data/song_data'

In [5]:
filepath = get_files(song_files)

In [6]:
# take a neel peak at our file list
filepath[0]

'/home/patrick/Documents/Semicolon/Projects/sparkify_data_engineering/virtual_env_sparkify/data/song_data/A/A/C/TRAACLV128F427E123.json'

In [9]:
first_file = filepath[0]

In [10]:
df = pd.read_json(first_file, lines=True)
df.head(3)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARDNS031187B9924F0,32.67828,-83.22295,Georgia,Tim Wilson,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,186.48771,2005


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
df.dtypes

num_songs             int64
artist_id            object
artist_latitude     float64
artist_longitude    float64
artist_location      object
artist_name          object
song_id              object
title                object
duration            float64
year                  int64
dtype: object

In [12]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SONYPOM12A8C13B2D7',
 'I Think My Wife Is Running Around On Me (Taco Hell)',
 'ARDNS031187B9924F0',
 2005,
 186.48771]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [13]:
song_table_insert

' INSERT INTO songs (\n                            song_id,title,\n                            artist_id,year, duration)\n                      VALUES (%s, %s, %s, %s, %s)\n                      ON CONFLICT DO NOTHING;\n'

In [14]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [15]:
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARDNS031187B9924F0,32.67828,-83.22295,Georgia,Tim Wilson,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,186.48771,2005


In [16]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['ARDNS031187B9924F0', 'Tim Wilson', 'Georgia', 32.67828, -83.22295]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [18]:
log_files = 'data/log_data'

In [19]:
filepath = get_files(log_files)

In [20]:
first_log_file = filepath[0]
first_log_file

'/home/patrick/Documents/Semicolon/Projects/sparkify_data_engineering/virtual_env_sparkify/data/log_data/2018/11/2018-11-16-events.json'

In [21]:
df2 = pd.read_json(first_log_file, lines=True)
df2.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mudhoney,Logged In,Aleena,F,10,Kirby,231.57506,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Get Into Yours,200,1542326457796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
1,Carpenters,Logged In,Aleena,F,11,Kirby,238.39302,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Yesterday Once More,200,1542326688796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
2,None,Logged In,Aleena,F,12,Kirby,NaN,paid,"Waterloo-Cedar Falls, IA",GET,Home,1.541023e+12,637,None,200,1542326691796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,Kings Of Leon,Logged In,Aleena,F,13,Kirby,185.28608,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Taper Jean Girl,200,1542326926796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,The Bloody Beetroots,Logged In,Aleena,F,14,Kirby,201.97832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Warp 1.9 (feat. Steve Aoki),200,1542327111796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [22]:
df2 =  df2.query('page == "NextSong"')
df2.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mudhoney,Logged In,Aleena,F,10,Kirby,231.57506,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Get Into Yours,200,1542326457796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
1,Carpenters,Logged In,Aleena,F,11,Kirby,238.39302,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Yesterday Once More,200,1542326688796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,Kings Of Leon,Logged In,Aleena,F,13,Kirby,185.28608,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Taper Jean Girl,200,1542326926796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,The Bloody Beetroots,Logged In,Aleena,F,14,Kirby,201.97832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Warp 1.9 (feat. Steve Aoki),200,1542327111796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
5,Kings Of Leon,Logged In,Aleena,F,15,Kirby,201.79546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Revelry,200,1542327312796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


In [25]:
t = pd.to_datetime(df2.ts, unit='ms')
t.head()

0   2018-11-16 00:00:57.796
1   2018-11-16 00:04:48.796
3   2018-11-16 00:08:46.796
4   2018-11-16 00:11:51.796
5   2018-11-16 00:15:12.796
Name: ts, dtype: datetime64[ns]

In [33]:
df2.shape

(319, 18)

In [37]:
time_data = {'start_time': t,
             'hour': t.dt.hour, 
             'day':t.dt.day,
             'week_of_year': t.dt.weekofyear,
             'month':t.dt.month,
             'year':t.dt.year,
             'weekday':t.dt.weekday}
# column_labels = ['Hour', 'Day', 'Month', 'Year', 'Weekday']

In [38]:
time_df = pd.DataFrame(time_data)
time_df.head()

,start_time,hour,day,week_of_year,month,year,weekday
0,2018-11-16 00:00:57.796,0,16,46,11,2018,4
1,2018-11-16 00:04:48.796,0,16,46,11,2018,4
3,2018-11-16 00:08:46.796,0,16,46,11,2018,4
4,2018-11-16 00:11:51.796,0,16,46,11,2018,4
5,2018-11-16 00:15:12.796,0,16,46,11,2018,4


## 

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [39]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [90]:
df2.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mudhoney,Logged In,Aleena,F,10,Kirby,231.57506,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Get Into Yours,200,1542326457796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


In [91]:
user_df = df2[['userId','firstName', 'lastName', 'gender','level']]
user_df

,userId,firstName,lastName,gender,level
0,44,Aleena,Kirby,F,paid
1,44,Aleena,Kirby,F,paid
3,44,Aleena,Kirby,F,paid
4,44,Aleena,Kirby,F,paid
5,44,Aleena,Kirby,F,paid
...,...,...,...,...,...
375,49,Chloe,Cuevas,F,paid
376,49,Chloe,Cuevas,F,paid
378,49,Chloe,Cuevas,F,paid
380,49,Chloe,Cuevas,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [92]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

UniqueViolation: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(44) already exists.


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [59]:
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARDNS031187B9924F0,32.67828,-83.22295,Georgia,Tim Wilson,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,186.48771,2005


In [60]:
df2.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mudhoney,Logged In,Aleena,F,10,Kirby,231.57506,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Get Into Yours,200,1542326457796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


In [61]:
### songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

In [62]:
for index, row in df2.iterrows():
    
#     print(row.song, row.artist, row.length)
#     break

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    print(results)
    break
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

AmbiguousColumn: column reference "artist_id" is ambiguous
LINE 1: ...LECT song_id, artist_id FROM songs JOIN artists ON artist_id
                                                              ^


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [29]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.